### Это попытка заспидранить обучение легковесного классификатора дефектов через чач бота. Можно смело переписывать.

Оно что-то учит, то как-то плоховато и непонятно что. Но перед этим не помешает пересмотреть код генерации синтетики и иметь хорошие данные.

Подумать о кропе, так как случайный кроп для инференса нас не устроит...

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models import mobilenet_v2
from torchvision import transforms
import cv2
import random
import glob
from aaa_image_enhancement.image_defects_detection import DefectNames
from aaa_image_enhancement.image_utils import ImageConversions
from aaa_image_enhancement.synth_defects import ImageDistortions
from sklearn.metrics import f1_score
import numpy as np

# Set device
device = torch.device(
    "mps"
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)


# подумать о шафле
class DefectsDataset(Dataset):
    def __init__(self, image_paths, epoch_size=512):
        self.all_image_paths = image_paths
        self.epoch_size = epoch_size
        self.image_paths = []
        self.shuffle_and_select()
        self.transform = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.RandomHorizontalFlip(),
                # transforms.RandomRotation(10),
                # transforms.ColorJitter(
                #     brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
                # ),
                transforms.RandomCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def shuffle_and_select(self):
        # Randomly select a subset of image paths for this epoch
        if len(self.all_image_paths) > self.epoch_size:
            self.image_paths = random.sample(self.all_image_paths, self.epoch_size)
        else:
            self.image_paths = self.all_image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        while img.shape[0] < 224 or img.shape[1] < 224:
            index = random.randint(0, len(self.image_paths) - 1)
            img_path = self.image_paths[index]
            img = cv2.imread(img_path)

        # Randomly apply distortions
        if random.random() < 0.5:
            distorter = ImageDistortions(img)
            distortions = random.sample(list(DefectNames), 1)  # increase later
            distorted_img, applied_distortions = distorter.apply_distortions(
                distortions
            )
            img = distorted_img
            labels = [
                1 if defect in applied_distortions else 0 for defect in DefectNames
            ]
        else:
            labels = [0] * len(DefectNames)

        # Apply transforms
        img = self.transform(img)

        return img, torch.tensor(labels, dtype=torch.float32), img_path


# Load image paths
image_paths = glob.glob("../data/real_estate_images/*.jpg")

# Create dataset and dataloader
dataset = DefectsDataset(image_paths, epoch_size=512)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)  # no need

# Define the model
# model = mobilenet_v2(
#     num_classes=len(DefectNames)
# )
model = mobilenet_v2(  # do we need weights?
    weights=torchvision.models.MobileNet_V2_Weights
)
model.classifier[1] = nn.Linear(model.last_channel, len(DefectNames))
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## watch labels of real images

In [ ]:
import matplotlib.pyplot as plt
import torchvision

# Assuming dataset and dataloader are already defined and initialized
dataset = DefectsDataset(image_paths)
dataloader = DataLoader(
    dataset, batch_size=32, shuffle=True
)  # Ensure there's enough data


def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean  # denormalize
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title, fontsize=8)
    plt.axis("off")  # Turn off axis numbers and ticks


def process_image(inp):
    """Process a Tensor for displaying."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean  # denormalize
    inp = np.clip(inp, 0, 1)
    return inp


# Assuming dataset and dataloader are already defined and initialized
dataset = DefectsDataset(image_paths)
dataloader = DataLoader(
    dataset, batch_size=32, shuffle=True
)  # Ensure there's enough data

# Get a batch of training data
images, labels, _ = next(iter(dataloader))

# Get random indices to display 9 images
indices = torch.randperm(images.size(0))[:9]

# Prepare titles with defect names
titles = [
    ", ".join(
        [
            defect.name
            for defect, present in zip(DefectNames, labels[idx])
            if present == 1
        ]
    )
    for idx in indices
]

# Plot images in a 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(10, 10))
for i, (ax, idx) in enumerate(zip(axes.flatten(), indices)):
    img = process_image(images[idx])  # Process the image for display
    ax.imshow(img)  # Use Matplotlib's imshow
    ax.set_title(titles[i], fontsize=8)
    ax.axis("off")  # Hide axes

plt.tight_layout()
plt.show()

## train model

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    dataset.shuffle_and_select()  # Shuffle and select new images for this epoch
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

    model.train()
    for images, labels, paths in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluation and metrics calculation
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels, paths in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            predictions = torch.sigmoid(outputs)

            all_labels.append(labels.cpu().numpy())
            all_predictions.append(predictions.cpu().numpy())

    # Concatenate all batches
    all_labels = np.vstack(all_labels)
    all_predictions = np.vstack(all_predictions)

    # Calculate metrics for each class
    f1_scores = [
        f1_score(all_labels[:, i], all_predictions[:, i] > 0.5)
        for i in range(len(DefectNames))
    ]
    average_f1_score = np.mean(f1_scores)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.2f}, Average F1 Score: {average_f1_score:.4f}, "
        + ", ".join(
            f"{defect.name} F1: {f1:.4f}" for defect, f1 in zip(DefectNames, f1_scores)
        )
    )

## Detect defects in real images

In [ ]:
from PIL import Image
from IPython.display import display

dataset = DefectsDataset(image_paths, epoch_size=10000)


def find_defective_images(model, dataset, defect_index, threshold=0.8, num_images=10):
    model.eval()
    # make some sort of map
    # defect_index = DefectNames.__members__[defect_name].value
    found_images = []
    found_paths = []  # List to save the paths of detected images

    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    with torch.no_grad():
        for images, labels, paths in dataloader:
            images = images.to(device)
            outputs = model(images)
            predictions = torch.sigmoid(outputs)

            probs = predictions[:, defect_index]
            indices = (probs > threshold).nonzero(as_tuple=True)[0]

            for idx in indices:
                if len(found_images) >= num_images:
                    break
                found_images.append(paths[idx])
                found_paths.append(paths[idx])  # Save the path

            if len(found_images) >= num_images:
                break

    # Display the found images
    for img_path in found_images:
        img = Image.open(img_path)
        display(img)

    return found_paths  # Return the list of paths for later review


detected_paths = find_defective_images(model, dataset, 0, threshold=0.8, num_images=10)